# Intro

Welcome! This script will demonstrate how to do some basic things with the model. You will need credentials to access the database, as well as all required python packages installed.

# Setup

To get set up, we need to set a few environment variables. You'll have to modify the "GOOGLE_APPLICATION_CREDENTIALS" variable to point to your own credentials file.


In [ ]:
# Note: you should fill in the appropriate values for the following environment variables. They aren't in the repo for security purposes
import os
os.environ['gcp_project'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = ""

We also want this notebook's working directory to be the root of this repo. This command will change the working directory if necessary

In [ ]:
if 'requirements.txt' not in os.listdir(os.getcwd()):
    os.chdir(os.path.join('..','..','..'))
os.getcwd()

And finally let's set up logging so that we can print messages to the screen and also to a file simultaneously

In [ ]:
import logging
from covid_model.utils import setup
outdir = setup('00000000_hello_world.ipynb')
logging.info("This is an example log message")

A copy of these logging messages (and more detailed messages not shown here by default) can be found in the `output\00000000_hello_world.ipynb` directory. Each time the `setup` function is run, a new logging file in that directory is created, with a name including the current timestamp.

In addition to your own logging messages, you will also see output from the model and various other functions, which are also logging their activity.

# Creating a model

To create a model, simply create an instance of the CovidModel class.

In [ ]:
from covid_model import CovidModel
m1 = CovidModel()

As you can see from the logging messages, the model automatically retrieved vaccination and hospitalization data from the database. __The data retrieved depend on the start and end dates of the model__, but since we didn't specify explicit start and end dates, default values are used.

## Setting the start and end dates of the model

Here are the default start and end dates of the model:

In [ ]:
logging.info(f'Start Date: {m1.start_date}')
logging.info(f'End Date: {m1.end_date}')

Internally, the model also numbers all the days between start and end date, where the start is always zero:

In [ ]:
logging.info(f'Start time: {m1.tstart}')
logging.info(f'End time: {m1.tend}')

Updating the end date will also change `tend` for consistency

In [ ]:
m1.end_date = '2022-10-01'
logging.info(f'End date: {m1.end_date}')
logging.info(f'End time: {m1.tend}')

Updating `tend` will also change the end date for consistency

In [ ]:
m1.tend = 1045
logging.info(f'End time: {m1.tend}')
logging.info(f'End date: {m1.end_date}')

Updating the start date _will not_ change `tstart`, because `tstart` is always zero. But it _will_ change `tend`, because a new date is serving as the starting point. Note that the end _date_ has not been changed.

In [ ]:
m1.start_date = '2020-01-24'
logging.info(f'End time: {m1.tend}')
logging.info(f'End date: {m1.end_date}')

Because `tstart` is always zero, it can't be changed manually:

In [ ]:
m1.tstart = 1

## Updating model data

If you change the start or end dates of the model, you'll probably have to reload the data to be consistent with the changes. Rather than automatically update data every time a change is made, data updating is done manually in order to minimize the number of queries to the database. This way a group of changes can be made and the update only needs to be performed once at the end.

In [ ]:
m1.update_data()

The update function only updates data that needs updating based on recent changes. If we run the function again, we see no logging output because no updates were necessary this time.

In [ ]:
m1.update_data()

In addition to the start and end dates, there are other model properties which also affect the data going into the model

## Other User-Defined Model Properties

Technically any property of the model can be manually set by the user, but the `CovidModel` class is designed to manage most of its properties on its own, leaving only a handful of properties for the user to worry about. Here is a list of properties the user is commonly responsible for setting appropriately:

* `start_date`: we discussed this already
* `end_date`: ditto
* `params_defs`: this should either be set as a Python dictionary or a path to a json file (which will be converted to a Python dictionary behind the scenes). This specifies the vast majority of parameters used in the model, such as the infectiousness of a particular variant, the incubation period, the risk of hospitalization for a particular age group, the effect of vaccination, etc.
* `region_defs`: this should either be set as a Python dictionary of a path to a json file (same as above). This defines the possible regions that the model can use, along with the county names and FIPS codes associated with each region.
* `regions`: Python list of strings indicating which regions to model. The region_defs dictionary must contain an entry for each region listed.
* `vacc_proj_params`: Again, either a Python dictionary or path to a json file. This defines how vaccinations should be projected into the future to fill in gaps for vaccination data.
* `mobility_mode`: specifies how to incorporate mobility in the model. For a single region, this doesn't matter. When modeling multiple regions at the same time, this determines how disease transmits between regions from mobility.
* `mobility_proj_params`: Python dictionary or json file. Specifes how mobility shoul dbe projected into the future to fill in gaps for mobility data. Currently this is a placeholder, as mobility projections aren't implemented yet.
* `hosp_reporting_frac`: A dictionary which enables changes in hospital reporting over time. The assumption is hospitals are reporting some fraction of people which are actually hospitalized for or with COVID-19. If the fraction is 1, then we assume all cases are reported. Anything less than 1 indicates underreporting, presumably due to testing protocols and reporting requirements at the hospital.


Let's look at some default values. 

The default `params_defs` and `region_defs` are quite verbose, but each has a common format that you should familiarize yourself with. More detail on `params_defs` in particular can be found in the README.md file in this repository.

In [ ]:
m1.params_defs

In [ ]:
m1.region_defs

By default, only one region is being modeled, the state of Colorado:

In [ ]:
m1.regions

These next three properties are all "none" by default, and need to be specified by the user if any vaccination or mobility projections are desired, or if the model needs to enable disease transmission across regions.

In [ ]:
m1.vacc_proj_params

In [ ]:
m1.mobility_mode

In [ ]:
m1.mobility_proj_params

Finally, the default hospital reporting fraction is set to 1 on January 24, 2020 and not changed for any later date. 

In [ ]:
m1.hosp_reporting_frac

This format of {date: value} is a common one that is also present in  `params_defs` as well as transmission control, `tc`. The idea is that values are set for a particular date, then assumed fixed until specified differently at a later date. So if we wanted to reflect a change in hospitalization reporting on May 2, 2020 reflecting a 20% drop in reported hospitalizations, we could do so like this:

`m1.hosp_reporting_frac = {'2020-01-24': 1, '2020-05-02': 0.8}`


## Creating a model with non-default properties

It is common to create a model and specify properties at the same time. Any model property can be passed as an argument to the CovidModel class. For instance, we could create a model with start and end dates, and specify the regions as well:

In [ ]:
m2 = CovidModel(start_date = '2020-01-24', end_date='2022-10-15', regions=['met', 'ms'])

It's also possible to specify all the non-default properties in a Python dictionary and pass that to the model with double asterisks (see [here](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)) like so:

In [ ]:
model_args = {
    'params_defs': 'covid_model/analysis/00000000_hello_world/params.json',
    'region_defs': 'covid_model/input/region_definitions.json',
    'vacc_proj_params': 'covid_model/input/vacc_proj_params.json',
    'start_date': '2020-01-24',
    'end_date': '2021-11-09',
    'max_step_size': 1.0,       # The biggest allowable step (in days) that the ODE solver is allowed to take.
    'ode_method': 'RK23'        # what method to use when solving the ODE
}
m3 = CovidModel(**model_args)

Notice there are a few properties that we haven't discussed yet, which relate to how the model solves the system of ODE's defining compartment dynamics.

## Defining a model using another model or from the database

Here are two other convenient ways of creating a model. First, you can specify a "base_model", which serves as the starting point for the new model:

In [ ]:
m4 = CovidModel(base_model=m3, end_date = '2022-10-15')

`m4` is exactly the same as `m3`, except that it has a different `end_date`, and also, data like vaccinations and mobility will also be different to reflect this new `end_date`.

Finally, we can load a model from the database, using `base_spec_id`, which specifies which model specifications should be loaded from the database. Side note: we frequently write models to the database after fitting them, so the fitted models can be easily loaded and run or re-fit as necessary.

In [ ]:
m5 = CovidModel(base_spec_id=2866, end_date = '2022-10-15')

# Prepping & Running the Model

Once you model, you solve the ODE's using the `solve_seir()` method. __However, before the model can be run, the parameters must be processed to produce the matrices that are necessary to solve the system of ODE's__. This critical step involves the model taking all of the parameter definitions, mobility & vaccination data, start & end dates, etc. and processing them into a series of vectors and matrices which capture the dynamics of the model at every point in time between `start_date` and `end_date`. The model will not be able to solve the ODE's without it.

Though the prep process is complicated behind the scenes, it's quite simple for you to do. Simply run the `prep()` method of the model you wish to build. Be patient, the this process can take a few (up to several) minutes to complete.

In [ ]:
m3.prep(outdir=outdir)
logging.info("Finished Prepping!")

After prepping the model, the only other necessary step is to set the _transmission control_, or TC, of the model, which controls the rate of transmission at any given time. This is done with the `update_tc()` method. Here we are specifying that at `t=0` (the first day), for the `'co'` regions, TC should be set to `0.75`. This value is implicitly fixed for all time since we don't specify any other values at any other `t`:

In [ ]:
m3.update_tc({0: {'co': 0.75}})

After prepping the model, you can now solve the ODE's to your heart's content. The key function here is `solve_seir()`:

In [ ]:
logging.info("Solving ODEs")
m3.solve_seir()
logging.info("solving done")

The result of running the above method is that we now have a numerical solution to the ODE's defining model dynamics. This means we now know how many people are in each compart between the start and end dates of the model. The solution is stored in `solution_y`, which is a Numpy array, where the rows are different days and the columns are different compartments. Here's a preview of that:

In [ ]:
logging.info(f'Solution shape: {m3.solution_y.shape}')
m3.solution_y[:5, :5]

A much more convenient way to inspect the solution is to look at `solution_ydf`, a Pandas dataframe holding the solution. Each column represents a different compartment, indicated by the column index, and each row is a different day, indicated by the row index. Here are the first 10 rows:

In [ ]:
m3.solution_ydf.head(10)

A subset of these columns tell us how many COVID-19 related hospitalizations there are. An easy way to look at the sum over all hospitalization compartments for each `t` is using the `solution_sum_Ih()` method, which returns a Numpy array:

In [ ]:
m3.solution_sum_Ih()

A more flexible method is `solution_sum_df()`, which can group together columns by any set of compartment attributes. This time, the result is a Pandas dataframe indexed by date. For instance, here's how we can get the totals for each disease status and each age group.

In [ ]:
m3.solution_sum_df(['seir', 'age']).head(10)

Another helpful method gives us the observed hospitalizations vs. the modeled hospitalizations. There are actual four columns, because the model is performing adjustments to account for the `hosp_reporting_frac` property mentioned earlier.

In [ ]:
hosps_df = m3.modeled_vs_observed_hosps()
hosps_df.head()

Note that the row index includes both region and date, since we track region-specific hospitalizations. Here is an explanation of each column:
* __observed__: The hospitalizations retrieved from the database without any adjustments
* __estimated_actual__: The result of dividing the __observed__ column by the hospitalization reporting fraction. This is our estimate of how many COVID-19 hospitalizations there actually are.
* __modeled_actual__: The total of all hospitalized compartments in the model. This will match the output of `solution_sum_Ih()`.
* __modeled_observed__: The result of multiplying the __modeled_actual__ column by the hospitalization reporting fraction. This is our modeled estimate of how many COVID-19 hospitalizations are/will be reported.

This is a common dataframe to plot, as it shows how well the model agrees with reality. Let's plot it:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure(figsize=(10, 5), dpi=300)
ax = fig.gca()
hosps_df.reset_index('region').drop(columns='region').plot(ax=ax)

First, since `hosp_reporting_frac` is always 1 for this model, the __observed__ and __estimated_actual__ columns will be the same, and the __modeled_actual__ and __modeled_observed__ columns will be the same. So some lines are being hidden behind other lines here.

Second, the answer to the question of how well the model agrees with reality is __not well__. This is why the model requires _fitting_, which we will discuss next.

# Fitting the Model

Fitting the model amounts to determining the _transmission control_ (TC) values which cause the modeled hospitalizations to agree with observed hospitalizations. TC can be specified at any day, but we have been fitting the model with TC being allowed to change every 14 days. Fitting is a process which involves the optimization of a function which inputs are TC values for different two-week periods, and output is a loss value capturing how much agreement there is between the observed and modeled hospitalizations over the time being modeled. The goal of fitting is to find the TC values which minimize the loss.

Practically, fitting takes place in several small batches, each batch covering a smaller amount of time and a few TC values. Batches may overlap, and batch characteristics are customizable, but an example of how batch fitting may look would involve the following steps:
1. Weeks 1-10 are fit, with TC changing every two weeks, meaning 5 TC values need to be fit. Hospitalizations from weeks 1-12 are used to compute the loss.
2. Weeks 5-14 are fit, meaning the two TC values for the first two weeks are "cemented" and considered part of the final fit.
3. Fitting continues in the sliding-window fashion until TC values for the entire fitting window have been fit.

Fitting can be done with the `do_single_fit` function in the `runnable_functions` module. Let's perform a fit on model `m3` (warning, this will take several minutes):

In [ ]:
from covid_model.runnable_functions import do_single_fit
m3_fit = do_single_fit(outdir=outdir, tc_window_batch_size=5, tc_batch_increment=2, write_results=False, prep_model=False, base_model=m3)

Now that the model has been fit, we can compare the modeled to observed hospitalizations again:

In [ ]:
hosps_df = m3_fit.modeled_vs_observed_hosps()
fig = plt.figure(figsize=(10, 5), dpi=300)
ax = fig.gca()
hosps_df.reset_index('region').drop(columns='region').plot(ax=ax)

As you can see, the modeled and fit hospitalizations are in much better agreement! Also, if you check the output directory you should see images and .csv files created for each batch fit.



# Running Model Reports

Having fit a model, it's often useful to generate some standard reports to understand the model output. The `do_create_report` method from the `covid_models.runnable_functions` module.

In [ ]:
from covid_model.runnable_functions import do_create_report
do_create_report(m3, outdir, immun_variants=['ba2121', 'ba2'], from_date='2021-01-01')
logging.info("report complete!")

Check the output directory to see the graphics that were produced there! By the way, the `runnable_functions` module contains other useful scripts for fitting several models either one at a time or in parallel, fitting several different scenerios, etc.

This concludes this brief but hopefully informative look at getting started with running our model. This is actually one of the few (if not the only) Jupyter notebook in this repo. For a more traditional look at what an analysis script might look like, take a look at `00000000_hello_world.py` in this directory as well, which will also introduce some other useful function in the `runnable_functions` module.

Best of luck in your modeling endeavors!